In [1]:
import itertools
import nltk
import operator
import nltk
nltk.download('wordnet')
from math import log
from matplotlib import pyplot as plt
from string import punctuation
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.probability import FreqDist
from collections import Counter

stopword_file = "Materials/stopwords.txt"
train_physics = "Materials/train/Physics.txt"
train_chem = "Materials/train/Chemistry.txt"
train_bio = "Materials/train/Biology.txt"

def read_file(fname):
    with open(fname) as f:
        content = f.readlines()                             # read file line by line into content
    content = [x.strip().lower() for x in content]          # remove starting/ending whitespaces and lowercase
    content = [nltk.word_tokenize(x) for x in content]      # tokenize the lines
    content = list(itertools.chain.from_iterable(content))  # join list of lists into single list
    words = [x for x in content if x not in punctuation      # remove punctuation from list
            and x != '--' and x != '"' and x != '”' and x != '“' and x != "''" and x != "``"]   # with some adjustments
    return [word for word in words if word.isalpha()]

def remove_stopwords(words):
    stopwords = read_file(stopword_file)
    return [word for word in words if word                  # removing words from stopwords.txt 
            not in stopwords] 

def lemmatization_stemming(words):
    lemmatized_words = [nltk.stem.WordNetLemmatizer().lemmatize(word) 
                        for word in words]                  # lemmatizing
    #print(lemmatized_words)
    return [nltk.stem.porter.PorterStemmer().stem(lemmatized_word) 
            for lemmatized_word in lemmatized_words]        # Stemming

def get_freq_dist(words):                                   # return frequency distribution
    return nltk.FreqDist(words)

def classify_based_on_prior(physics_prior, chem_prior, bio_prior): # classify a document based on highest prior
    if(physics_prior > chem_prior and physics_prior > bio_prior):
        return "Physics"
    if(chem_prior > physics_prior and chem_prior > bio_prior):
        return "Chemistry"
    if(bio_prior > chem_prior and bio_prior > physics_prior):
        return "Biology"
    
def compute_probability(trainModel, testItems, base = 2): # Function to compute the Conditional probability for a given class
    loglikelihood = 0
    for item, freq in testItems.items():  # log(0) = 0 only for unknown elements (don't use in practice)
        log_p_w_h = log(trainModel[item], base) if item in trainModel and trainModel[item] > 0 else 0
        loglikelihood += freq * log_p_w_h
        #Check if division by p(x) needed
    return loglikelihood

def lidstone_smoothing(vocabulary_list, keys, alpha):# function to calculated the lidstone smoothing for a vocabulary distribution
    n = sum(vocabulary_list.values()) 
    lidstone_vocabulary_list = dict(vocabulary_list)
    
    for key in keys:
        if key not in vocabulary_list.keys():
            lidstone_vocabulary_list.update({key:0})         
    
    v = len(lidstone_vocabulary_list.keys())
    
    for key, value in lidstone_vocabulary_list.items():
        ls_prob = (value + alpha) / (n + (v * alpha))
        lidstone_vocabulary_list[key] = ls_prob
    
    return lidstone_vocabulary_list

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tifat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
physics_words = read_file(train_physics)                    # read files
chem_words = read_file(train_chem)
bio_words = read_file(train_bio)
print(len(physics_words), len(chem_words), len(bio_words))

physics_words = remove_stopwords(physics_words)             # stopword removal 
chem_words = remove_stopwords(chem_words)
bio_words = remove_stopwords(bio_words)
print(len(physics_words), len(chem_words), len(bio_words))

physics_words = lemmatization_stemming(physics_words)       # lemmatization & stemming 
chem_words = lemmatization_stemming(chem_words)
bio_words = lemmatization_stemming(bio_words)
print(len(physics_words), len(chem_words), len(bio_words))


#print(physics_words)

99745 109202 170683
36935 49346 77165
36935 49346 77165


In [4]:
# Getting word frequency 
physics_freq = get_freq_dist(physics_words)
chem_freq = get_freq_dist(chem_words)
bio_freq = get_freq_dist(bio_words)
combined_freq = Counter(physics_freq) + Counter(chem_freq) + Counter(bio_freq)
#print(physics_freq, chem_freq, bio_freq)

physics_kv = ((key,value) for key,value in physics_freq.items())
physics_keys = [x[0] for x in physics_kv]
chem_kv = ((key, value) for key,value in chem_freq.items())
chem_keys = [x[0] for x in chem_kv]
bio_kv = ((key,value) for key,value in bio_freq.items())
bio_keys = [x[0] for x in bio_kv]

#Smoothing the three documents
#physics_freq = lidstone_smoothing(physics_freq, chem_keys + bio_keys, 1)
#chem_freq = lidstone_smoothing(chem_freq, physics_keys + bio_keys, 1)
#bio_freq = lidstone_smoothing(bio_freq, chem_keys + physics_keys, 1)
#combined_freq = Counter(physics_freq) + Counter(chem_freq) + Counter(bio_freq)

# Calculating per class pmi
physics_pmi = {}
chem_pmi = {}
bio_pmi = {}

# Helper sums to prevent summing everytime
physics_count = sum(physics_freq.values())
chem_count = sum(chem_freq.values())
bio_count = sum(bio_freq.values())
combined_count = sum(combined_freq.values())

for key, value in physics_freq.items():
    word_count_overall = combined_freq[key]
    physics_pmi[key] = log((value/physics_count)/(word_count_overall/combined_count),2)

for key, value in chem_freq.items():
    word_count_overall = combined_freq[key]
    chem_pmi[key] = log((value/chem_count)/(word_count_overall/combined_count),2)

for key, value in bio_freq.items():
    word_count_overall = combined_freq[key]
    bio_pmi[key] = log((value/bio_count)/(word_count_overall/combined_count),2)


#print(physics_freq, chem_freq, bio_freq)
print(combined_count, physics_count, chem_count, bio_count)

# Sorting and picking top 30 for displaying
physics_sort = sorted(((v,k) for k,v in physics_pmi.items()), reverse=True)[:30]
chem_sort = sorted(((v,k) for k,v in chem_pmi.items()), reverse=True)[:30]
bio_sort = sorted(((v,k) for k,v in bio_pmi.items()), reverse=True)[:30]
print('\033[1m','\nPhysics PMI','\033[0m')
print(physics_sort)
print('\033[1m','\nChem PMI','\033[0m')
print(chem_sort)
print('\033[1m','\nBio PMI','\033[0m')
print(bio_sort)

163446 36935 49346 77165
 
Physics PMI 
[(2.145753587820799, 'whereof'), (2.145753587820799, 'warranti'), (2.145753587820799, 'void'), (2.145753587820799, 'vibrat'), (2.145753587820799, 'vehement'), (2.145753587820799, 'vapour'), (2.145753587820799, 'uncompound'), (2.145753587820799, 'twelfth'), (2.145753587820799, 'traject'), (2.145753587820799, 'trademark'), (2.145753587820799, 'tq'), (2.145753587820799, 'thinner'), (2.145753587820799, 'thereabout'), (2.145753587820799, 'theor'), (2.145753587820799, 'telescop'), (2.145753587820799, 'subtil'), (2.145753587820799, 'split'), (2.145753587820799, 'speculum'), (2.145753587820799, 'soonest'), (2.145753587820799, 'soever'), (2.145753587820799, 'shew'), (2.145753587820799, 'semicircular'), (2.145753587820799, 'secant'), (2.145753587820799, 'salin'), (2.145753587820799, 'ruler'), (2.145753587820799, 'royalti'), (2.145753587820799, 'refund'), (2.145753587820799, 'refrang'), (2.145753587820799, 'reflex'), (2.145753587820799, 'redistribut')]
 
Ch

In [5]:
# top 10 feature selection using pmi
physics_features = {}
chem_features = {}
bio_features = {}

for key, value in physics_pmi.items():
    if(key in chem_pmi):
        if(chem_pmi[key] > value):
            continue
    if(key in bio_pmi):
        if(bio_pmi[key] > value):
            continue
    physics_features[key] = value

for key, value in chem_pmi.items():
    if(key in physics_pmi):
        if(physics_pmi[key] > value):
            continue
    if(key in bio_pmi):
        if(bio_pmi[key] > value):
            continue
    chem_features[key] = value
    
for key, value in bio_pmi.items():
    if(key in chem_pmi):
        if(chem_pmi[key] > value):
            continue
    if(key in physics_pmi):
        if(physics_pmi[key] > value):
            continue
    bio_features[key] = value

print('\033[1m','Physics:','\033[0m', len(physics_freq), len(physics_features)) 
print('\033[1m','Chemistry:','\033[0m', len(chem_freq), len(chem_features))
print('\033[1m','Biology:','\033[0m', len(bio_freq), len(bio_features))

physics_final = sorted(((v,k) for k,v in physics_features.items()), reverse=True)[:10]
chem_final = sorted(((v,k) for k,v in chem_features.items()), reverse=True)[:10]
bio_final = sorted(((v,k) for k,v in bio_features.items()), reverse=True)[:10]
print('\033[1m','\nPhysics 10 features using pmi','\033[0m')
print(physics_final)
print('\033[1m','\nChem 10 features using pmi','\033[0m')
print(chem_final)
print('\033[1m','\nBio 10 features using pmi','\033[0m')
print(bio_final)

 Physics:  2697 1760
 Chemistry:  3104 1902
 Biology:  7099 5744
 
Physics 10 features using pmi 
[(2.145753587820799, 'whereof'), (2.145753587820799, 'warranti'), (2.145753587820799, 'void'), (2.145753587820799, 'vibrat'), (2.145753587820799, 'vehement'), (2.145753587820799, 'vapour'), (2.145753587820799, 'uncompound'), (2.145753587820799, 'twelfth'), (2.145753587820799, 'traject'), (2.145753587820799, 'trademark')]
 
Chem 10 features using pmi 
[(1.7278090212979142, 'sulphid'), (1.7278090212979142, 'sulphat'), (1.7278090212979142, 'strontium'), (1.7278090212979142, 'solvent'), (1.7278090212979142, 'sodium'), (1.7278090212979142, 'soda'), (1.7278090212979142, 'silic'), (1.7278090212979142, 'pb'), (1.7278090212979142, 'ozon'), (1.7278090212979142, 'oxyhydrogen')]
 
Bio 10 features using pmi 
[(1.082795537876925, 'zululand'), (1.082795537876925, 'zoolog'), (1.082795537876925, 'zool'), (1.082795537876925, 'zoo'), (1.082795537876925, 'zone'), (1.082795537876925, 'zenith'), (1.082795537876

In [6]:
# Mutual Information per class
# I(x;C) = p(x,C) log(p(x,C)/(p(x)p(C)))

# Calculate per class MI
physics_mi = {}
chem_mi = {}
bio_mi = {}

for key, value in physics_freq.items():
    word_count_overall = combined_freq[key]
    physics_mi[key] = (value/physics_count) * log((value/physics_count)/((word_count_overall/combined_count) * (physics_count/combined_count)),2)
    
for key, value in chem_freq.items():
    word_count_overall = combined_freq[key]
    chem_mi[key] = (value/chem_count) * log((value/chem_count)/((word_count_overall/combined_count) * (chem_count/combined_count)),2)

for key, value in bio_freq.items():
    word_count_overall = combined_freq[key]
    bio_mi[key] = (value/bio_count) * log((value/bio_count)/((word_count_overall/combined_count) * (bio_count/combined_count)),2)

# Sorting and picking top 30 for displaying
physics_mi_sort = sorted(((v,k) for k,v in physics_mi.items()), reverse=True)[:30]
chem_mi_sort = sorted(((v,k) for k,v in chem_mi.items()), reverse=True)[:30]
bio_mi_sort = sorted(((v,k) for k,v in bio_mi.items()), reverse=True)[:30]
print('\033[1m','\nPhysics MI','\033[0m')
print(physics_mi_sort)
print('\033[1m','\nChem MI','\033[0m')
print(chem_mi_sort)
print('\033[1m','\nBio MI','\033[0m')
print(bio_mi_sort)

 
Physics MI 
[(0.11452508761979771, 'colour'), (0.0933088340828989, 'light'), (0.08725840086977399, 'ray'), (0.08400519973940627, 'refract'), (0.048967475440655224, 'part'), (0.04577895203462142, 'prism'), (0.044286886356212474, 'glass'), (0.042612878704871814, 'red'), (0.03640481088017027, 'bodi'), (0.03485755687099164, 'reflect'), (0.03475009370237919, 'distanc'), (0.03294097092728653, 'white'), (0.03141209619604611, 'ring'), (0.030083012015074573, 'blue'), (0.028036530113189843, 'paper'), (0.025983647583483097, 'inch'), (0.024495292516114958, 'side'), (0.024361479670074918, 'incid'), (0.02417845035553442, 'made'), (0.022889587480747115, 'refrang'), (0.02234551549121573, 'yellow'), (0.021236767249386578, 'violet'), (0.02077983433908203, 'angl'), (0.020673874735776237, 'observ'), (0.019636245097696764, 'reflexion'), (0.01913601601106203, 'imag'), (0.018842794375582273, 'sun'), (0.01877917587683094, 'green'), (0.01870671870253952, 'sine'), (0.018449096994596903, 'experi')]
 
Chem MI 


In [7]:
# top 10 features per class using MI

physics_mi_features = {}
chem_mi_features = {}
bio_mi_features = {}

for key, value in physics_mi.items():
    if(key in chem_mi):
        if(chem_mi[key] > value):
            continue
    if(key in bio_mi):
        if(bio_mi[key] > value):
            continue
    physics_mi_features[key] = value

for key, value in chem_mi.items():
    if(key in physics_mi):
        if(physics_mi[key] > value):
            continue
    if(key in bio_mi):
        if(bio_mi[key] > value):
            continue
    chem_mi_features[key] = value
    
for key, value in bio_mi.items():
    if(key in chem_mi):
        if(chem_mi[key] > value):
            continue
    if(key in physics_mi):
        if(physics_mi[key] > value):
            continue
    bio_mi_features[key] = value

print('\033[1m','Physics:','\033[0m', len(physics_freq), len(physics_mi_features)) 
print('\033[1m','Chemistry:','\033[0m', len(chem_freq), len(chem_mi_features))
print('\033[1m','Biology:','\033[0m', len(bio_freq), len(bio_mi_features))

physics_mi_final = sorted(((v,k) for k,v in physics_mi_features.items()), reverse=True)[:10]
chem_mi_final = sorted(((v,k) for k,v in chem_mi_features.items()), reverse=True)[:10]
bio_mi_final = sorted(((v,k) for k,v in bio_mi_features.items()), reverse=True)[:10]
print('\033[1m','\nPhysics 10 features using MI','\033[0m')
print(physics_mi_final)
print('\033[1m','\nChem 10 features using MI','\033[0m')
print(chem_mi_final)
print('\033[1m','\nBio 10 features using MI','\033[0m')
print(bio_mi_final)
#print(physics_mi_features['colour'])

 Physics:  2697 1806
 Chemistry:  3104 1922
 Biology:  7099 5678
 
Physics 10 features using MI 
[(0.11452508761979771, 'colour'), (0.0933088340828989, 'light'), (0.08725840086977399, 'ray'), (0.08400519973940627, 'refract'), (0.048967475440655224, 'part'), (0.04577895203462142, 'prism'), (0.044286886356212474, 'glass'), (0.042612878704871814, 'red'), (0.03640481088017027, 'bodi'), (0.03485755687099164, 'reflect')]
 
Chem 10 features using MI 
[(0.07138825213642247, 'acid'), (0.05270944340566383, 'form'), (0.04657279351520724, 'water'), (0.04040634723336831, 'oxygen'), (0.038585610616266794, 'carbon'), (0.03564442069633358, 'hydrogen'), (0.03422192975175914, 'element'), (0.032475746899214425, 'ga'), (0.03171973921598434, 'compound'), (0.03130266414786072, 'oxid')]
 
Bio 10 features using MI 
[(0.03864470823965595, 'game'), (0.034790584558882474, 'bird'), (0.02420424729800839, 'state'), (0.022507665946408184, 'wild'), (0.015566492907100143, 'kill'), (0.015241206194005427, 'law'), (0.014

In [8]:
physics_pmi_final_words = [x[1] for x in physics_final]
chem_pmi_final_words = [x[1] for x in chem_final]
bio_pmi_final_words = [x[1] for x in bio_final]

print('\033[1m','\nPMI Features','\033[0m')
print('Physics', physics_pmi_final_words)
print('Chemistry', chem_pmi_final_words)
print('Biology', bio_pmi_final_words)

physics_mi_final_words = [x[1] for x in physics_mi_final]
chem_mi_final_words = [x[1] for x in chem_mi_final]
bio_mi_final_words = [x[1] for x in bio_mi_final]

print('\033[1m','\nMI Features','\033[0m')
print('Physics', physics_mi_final_words)
print('Chemistry', chem_mi_final_words)
print('Biology', bio_mi_final_words)

test1 = "Materials/test/test1.txt"
test2 = "Materials/test/test2.txt"
test3 = "Materials/test/test3.txt"
test4 = "Materials/test/test4.txt"
test5 = "Materials/test/test5.txt"
test6 = "Materials/test/test6.txt"

test1_words = read_file(test1)
test1_words = remove_stopwords(test1_words)
test1_words = lemmatization_stemming(test1_words)
test1_freq = get_freq_dist(test1_words)

test2_words = read_file(test2)
test2_words = remove_stopwords(test2_words)
test2_words = lemmatization_stemming(test2_words)
test2_freq = get_freq_dist(test2_words)

test3_words = read_file(test3)
test3_words = remove_stopwords(test3_words)
test3_words = lemmatization_stemming(test3_words)
test3_freq = get_freq_dist(test3_words)

test4_words = read_file(test4)
test4_words = remove_stopwords(test4_words)
test4_words = lemmatization_stemming(test4_words)
test4_freq = get_freq_dist(test4_words)

test5_words = read_file(test5)
test5_words = remove_stopwords(test5_words)
test5_words = lemmatization_stemming(test5_words)
test5_freq = get_freq_dist(test5_words)

test6_words = read_file(test6)
test6_words = remove_stopwords(test6_words)
test6_words = lemmatization_stemming(test6_words)
test6_freq = get_freq_dist(test6_words)

 
PMI Features 
Physics ['whereof', 'warranti', 'void', 'vibrat', 'vehement', 'vapour', 'uncompound', 'twelfth', 'traject', 'trademark']
Chemistry ['sulphid', 'sulphat', 'strontium', 'solvent', 'sodium', 'soda', 'silic', 'pb', 'ozon', 'oxyhydrogen']
Biology ['zululand', 'zoolog', 'zool', 'zoo', 'zone', 'zenith', 'zebra', 'zealand', 'yunnan', 'yukon']
 
MI Features 
Physics ['colour', 'light', 'ray', 'refract', 'part', 'prism', 'glass', 'red', 'bodi', 'reflect']
Chemistry ['acid', 'form', 'water', 'oxygen', 'carbon', 'hydrogen', 'element', 'ga', 'compound', 'oxid']
Biology ['game', 'bird', 'state', 'wild', 'kill', 'law', 'protect', 'speci', 'year', 'preserv']


In [9]:
# selecting top 30 features, 10 each for phy, chem and bio for both pmi and mi
physics_features_copy = physics_features
chem_features_copy = chem_features
bio_features_copy = bio_features

physics_features = {key:physics_features_copy[key] for key in physics_pmi_final_words}
chem_features = {key:chem_features_copy[key] for key in chem_pmi_final_words}
bio_features = {key:bio_features_copy[key] for key in bio_pmi_final_words}

physics_mi_features_copy = physics_mi_features
chem_mi_features_copy = chem_mi_features
bio_mi_features_copy = bio_mi_features

physics_mi_features = {key:physics_mi_features_copy[key] for key in physics_mi_final_words}
chem_mi_features = {key:chem_mi_features_copy[key] for key in chem_mi_final_words}
bio_mi_features = {key:bio_mi_features_copy[key] for key in bio_mi_final_words}

#print(physics_features, chem_features, bio_features)
#print(physics_mi_features, chem_mi_features, bio_mi_features)

# test with pmi features
test1_physics = compute_probability(physics_features, test1_freq)
test2_physics = compute_probability(physics_features, test2_freq)
test3_physics = compute_probability(physics_features, test3_freq)
test4_physics = compute_probability(physics_features, test4_freq)
test5_physics = compute_probability(physics_features, test5_freq)
test6_physics = compute_probability(physics_features, test6_freq)

test1_chem = compute_probability(chem_features, test1_freq)
test2_chem = compute_probability(chem_features, test2_freq)
test3_chem = compute_probability(chem_features, test3_freq)
test4_chem = compute_probability(chem_features, test4_freq)
test5_chem = compute_probability(chem_features, test5_freq)
test6_chem = compute_probability(chem_features, test6_freq)

test1_bio = compute_probability(bio_features, test1_freq)
test2_bio = compute_probability(bio_features, test2_freq)
test3_bio = compute_probability(bio_features, test3_freq)
test4_bio = compute_probability(bio_features, test4_freq)
test5_bio = compute_probability(bio_features, test5_freq)
test6_bio = compute_probability(bio_features, test6_freq)

# test with mi features
test1_mi_physics = compute_probability(physics_mi_features, test1_freq)
test2_mi_physics = compute_probability(physics_mi_features, test2_freq)
test3_mi_physics = compute_probability(physics_mi_features, test3_freq)
test4_mi_physics = compute_probability(physics_mi_features, test4_freq)
test5_mi_physics = compute_probability(physics_mi_features, test5_freq)
test6_mi_physics = compute_probability(physics_mi_features, test6_freq)

test1_mi_chem = compute_probability(chem_mi_features, test1_freq)
test2_mi_chem = compute_probability(chem_mi_features, test2_freq)
test3_mi_chem = compute_probability(chem_mi_features, test3_freq)
test4_mi_chem = compute_probability(chem_mi_features, test4_freq)
test5_mi_chem = compute_probability(chem_mi_features, test5_freq)
test6_mi_chem = compute_probability(chem_mi_features, test6_freq)

test1_mi_bio = compute_probability(bio_mi_features, test1_freq)
test2_mi_bio = compute_probability(bio_mi_features, test2_freq)
test3_mi_bio = compute_probability(bio_mi_features, test3_freq)
test4_mi_bio = compute_probability(bio_mi_features, test4_freq)
test5_mi_bio = compute_probability(bio_mi_features, test5_freq)
test6_mi_bio = compute_probability(bio_mi_features, test6_freq)



# prediction
print('\033[1m','\nPrediction using PMI','\033[0m')
print('test1.txt: classified as', classify_based_on_prior(test1_physics, test1_chem, test1_bio))
print('test2.txt: classified as', classify_based_on_prior(test2_physics, test2_chem, test2_bio))
print('test3.txt: classified as', classify_based_on_prior(test3_physics, test3_chem, test3_bio))
print('test4.txt: classified as', classify_based_on_prior(test4_physics, test4_chem, test4_bio))
print('test5.txt: classified as', classify_based_on_prior(test5_physics, test5_chem, test5_bio))
print('test6.txt: classified as', classify_based_on_prior(test6_physics, test6_chem, test6_bio))

print('\033[1m','\nPrediction using MI', '\033[0m')
print('test1.txt: classified as', classify_based_on_prior(test1_mi_physics, test1_mi_chem, test1_mi_bio))
print('test2.txt: classified as', classify_based_on_prior(test2_mi_physics, test2_mi_chem, test2_mi_bio))
print('test3.txt: classified as', classify_based_on_prior(test3_mi_physics, test3_mi_chem, test3_mi_bio))
print('test4.txt: classified as', classify_based_on_prior(test4_mi_physics, test4_mi_chem, test4_mi_bio))
print('test5.txt: classified as', classify_based_on_prior(test5_mi_physics, test5_mi_chem, test5_mi_bio))
print('test6.txt: classified as', classify_based_on_prior(test6_mi_physics, test6_mi_chem, test6_mi_bio))


 
Prediction using PMI 
test1.txt: classified as Biology
test2.txt: classified as Physics
test3.txt: classified as Physics
test4.txt: classified as Chemistry
test5.txt: classified as Chemistry
test6.txt: classified as Physics
 
Prediction using MI 
test1.txt: classified as Chemistry
test2.txt: classified as Physics
test3.txt: classified as Biology
test4.txt: classified as Physics
test5.txt: classified as Biology
test6.txt: classified as Biology
